In [1]:
%load_ext google.cloud.bigquery

In [2]:
import branca
import altair as alt
import autocensus
import folium
import geopandas as gpd
import pandas as pd
import requests

## Transportation and residential energy use are major drivers of climate change

In Virginia, transportation and residential energy use together account for about half of greenhouse gas (GHG) emissions, at 29.4% and 16.8%, respectively. Reducing our carbon footprint will require reducing emissions from driving and home energy use. Data from https://www.deq.virginia.gov/air/greenhouse-gases.

In [3]:
va_eia_raw = requests.get("https://www.deq.virginia.gov/home/showpublisheddocument/12632/637725680591230000").content
va_eia_by_source = pd.read_excel(va_eia_raw, sheet_name="Figure 1")

In [4]:
alt.Chart(va_eia_by_source).mark_bar().encode(
    y=alt.X("Sector:N", sort="-x"),
    x=alt.Y("Percent of Total Emissions:Q"),
    color="Sector:N",
    tooltip="Percent of Total Emissions:Q",
).properties(
    title="Virginia Emissions by Sector, 2018",
)

alt.Chart(...)

## Multi-family homes use less energy than single-family homes.

Using data from the US Energy Information Administration, we can compare energy use and energy cost per household across housing unit types. See https://www.eia.gov/consumption/residential/data/2015/index.php?view=consumption for source data; results are from 2015. Single-family homes are the most energy-intensive, with apartments of five or more units consuming about one third the energy of a single-family home.

In [5]:
%%bigquery hh_energy_use
select
    *
from `cvilledata.eia.recs_2015_ce1_unit_type`

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.82rows/s]


In [6]:
alt.Chart(hh_energy_use).mark_bar().encode(
    y=alt.X("housing_unit_type:N", sort="-x"),
    x=alt.Y("per_household_million_btu:Q"),
    color="housing_unit_type:N",
    tooltip="per_household_million_btu:Q",
).properties(
    title="Average household energy use by housing type",
)

alt.Chart(...)

In [7]:
alt.Chart(hh_energy_use).mark_bar().encode(
    y=alt.X("housing_unit_type:N", sort="-x"),
    x=alt.Y("per_household_dollars:Q"),
    color="housing_unit_type:N",
    tooltip="per_household_dollars:Q",
).properties(
    title="Average household energy costs by housing type",
)

alt.Chart(...)

## City residents drive less than County residents.

We can use data from the American Community Survey to estimate the proportion of workers who commute by car (prop_drive) and the average commuting time (travel_time) for the Charlottesville MSA. Data are from 2019. Relative to Albemarle, Fluvanna, Greene, and Nelson counties, Charlottesville residents are less likely to drive to work, and spend much less time commuting.

In [8]:
MODE_SHARE_VARIABLES = [
    # Sex of Workers by Means of Transportation to Work
    "B08006_001E",  # Estimate!!Total
    "B08006_002E",  # Estimate!!Total!!Car, truck, or van
    "B08006_017E",  # Estimate!!Total!!Worked at home
]

TRAVEL_TIME_VARIABLES = [
    # Aggregate Travel Time to Work of Workers by Sex
    "B08013_001E",  # Estimate!!Aggregate travel time to work (in minutes)
    # Sex of Workers by Means of Transportation to Work
    "B08006_001E",  # Estimate!!Total
    "B08006_017E",  # Estimate!!Total!!Worked at home
]

CVILLE_MSA_COUNTIES = {
    "003": "Albemarle County",
    "540": "Charlottesville City",
    "065": "Fluvanna County",
    "079": "Greene County",
    "125": "Nelson County",
}

def reshape_census(df):
    pivoted = df.pivot(index=["name"], columns="variable_code", values="value").reset_index()
    geo_deduped = df[["name", "geometry"]].drop_duplicates()
    return pd.merge(pivoted, geo_deduped, on="name")

def normalize_county(df, raw_county_key="name", normalized_county_key="county"):
    df = df.copy()
    df[normalized_county_key] = df[raw_county_key].str.replace(" city", " City").str.replace(", Virginia", "")
    return df

def calculate_mode_share(df):
    df = df.copy()
    df["prop_drive"] = df["B08006_002E"] / (df["B08006_001E"] - df["B08006_017E"])
    return df

def calculate_travel_time(df):
    df = df.copy()
    df["travel_time"] = df["B08013_001E"] / (df["B08006_001E"] - df["B08006_017E"])
    return df

In [9]:
commute_mode_county = autocensus.Query(
    estimate=5,
    years=[2019],
    variables=MODE_SHARE_VARIABLES,
    for_geo=["county:*"],
    in_geo=["state:51"],  # Virginia
    geometry="polygons",
).run()
commute_mode_county_wide = calculate_mode_share(normalize_county(reshape_census(commute_mode_county)))

Retrieving variables...
Retrieving ACS tables...
Retrieving shapefiles...
Merging ACS tables and variables...
Merging annotations...
Merging shapefiles...
Finalizing data...


In [10]:
commute_mode_county_bar = commute_mode_county_wide.drop(labels="geometry", axis=1)
alt.Chart(commute_mode_county_bar[commute_mode_county_bar["county"].isin(CVILLE_MSA_COUNTIES.values())]).mark_bar().encode(
    y=alt.X("county:N", sort="-x"),
    x=alt.Y("prop_drive:Q"),
    color="county:N",
    tooltip="prop_drive:Q",
).properties(
    title="Proportion of residents who drive to work",
)

alt.Chart(...)

In [11]:
commute_mode_tract = autocensus.Query(
    estimate=5,
    years=[2019],
    variables=MODE_SHARE_VARIABLES,
    for_geo=["tract:*"],
    in_geo=["state:51", "county:540"],  # Virginia | Charlottesville City
    geometry="polygons",
).run()
commute_mode_tract_wide = calculate_mode_share(normalize_county(reshape_census(commute_mode_tract)))

Retrieving variables...
Retrieving ACS tables...
Retrieving shapefiles...
Merging ACS tables and variables...
Merging annotations...
Merging shapefiles...
Finalizing data...


In [12]:
m = folium.Map(
    location=[38.04, -78.49],
    tiles="cartodbpositron",
    zoom_start=13.75,
    zoom_snap=0.25,
    zoom_delta=0.25,
)
prop_drive = commute_mode_tract_wide["prop_drive"]
colormap = branca.colormap.linear.YlOrRd_09.scale(prop_drive.min(), prop_drive.max())
folium.GeoJson(
    gpd.GeoDataFrame(commute_mode_tract_wide)._to_geo(),
    style_function=lambda feature: {
        "color": "black",
        "fillOpacity": 0.75,
        "fillColor": colormap.rgb_hex_str(feature["properties"]["prop_drive"]),
    },
    tooltip=folium.GeoJsonTooltip(fields=["prop_drive"], labels=True),
).add_to(m)
legend = colormap.to_step(n=5, data=commute_mode_tract_wide["prop_drive"])
legend.caption = 'prop_drive'
legend.add_to(m)
m

In [13]:
travel_time_county = autocensus.Query(
    estimate=5,
    years=[2019],
    variables=TRAVEL_TIME_VARIABLES,
    for_geo=["county:*"],
    in_geo=["state:51"],  # Virginia
    geometry="polygons",
).run()

Retrieving variables...
Retrieving ACS tables...
Retrieving shapefiles...
Merging ACS tables and variables...
Merging annotations...
Merging shapefiles...
Finalizing data...


In [14]:
travel_time_county_wide = calculate_travel_time(normalize_county(reshape_census(travel_time_county)))

In [15]:
travel_time_county_bar = travel_time_county_wide.drop(labels="geometry", axis=1)
alt.Chart(travel_time_county_bar[travel_time_county_bar["county"].isin(CVILLE_MSA_COUNTIES.values())]).mark_bar().encode(
    y=alt.X("county:N", sort="-x"),
    x=alt.Y("travel_time:Q"),
    color="county:N",
    tooltip="travel_time:Q",
).properties(
    title="Average travel time to work by county",
)

alt.Chart(...)

## City households produce lower GHG emissions than County households.

We can use data from the Cool Climate survey to estimate per-households GHG emissions for the Charlottesville MSA. See https://coolclimate.org/ for source data; results are from 2014. Households in Charlottesville produce lower GHG emissions than the surrounding counties, and this difference is mostly explained by lower emissions from transportation and residential energy use.

In [16]:
%%bigquery emissions
select
    *
from `cvilledata.cool_climate.county`
where state = "VA" and county in ("CHARLOTTESVILLE CITY", "ALBEMARLE", "FLUVANNA", "GREENE", "NELSON")

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.99rows/s]


In [17]:
alt.Chart(emissions).mark_bar().encode(
    y=alt.X("county:N", sort="-x"),
    x=alt.Y("total_household_carbon_footprint_tco_e_yr:Q"),
    color="county:N",
    tooltip="total_household_carbon_footprint_tco_e_yr:Q",
).properties(
    title="Average household emissions by county",
)

alt.Chart(...)

In [18]:
emissions_long = emissions.melt(
    id_vars=["county"],
    value_vars=[
        "transport_tco_e_yr",
        "housing_tco_e_yr",
        "food_tco_e_yr",
        "goods_tco_e_yr",
        "services_tco_e_yr",
    ],
    var_name="category",
    value_name="tco_e_yr",
)
emissions_long["category"] = emissions_long["category"].str.replace("_tco_e_yr", "")

In [19]:
alt.Chart(emissions_long).mark_bar().encode(
    y=alt.X("county:N", sort="-x"),
    x=alt.Y("tco_e_yr:Q"),
    color="category:N",
    tooltip="tco_e_yr:Q",
).properties(
    title="Average household emissions by county and source",
)

alt.Chart(...)